# T5 untrained pretraining with div color dataset and custom tokenizer

2024-13-03 18:24

Testing using a cstom-ra

In [1]:
import shutil
!pip install -U git+https://github.com/huggingface/transformers.git@13493215abceafc1653af88b045120014fb4c1fc
!pip install -U git+https://github.com/huggingface/accelerate.git@8ade23cc6aec7c3bd3d80fef6378cafaade75bbe
!pip install -U git+https://github.com/huggingface/datasets.git@01f91bae037c98f2e05456287bab21470adb8f07
!pip install -U git+https://github.com/huggingface/evaluate.git@55f1bc6e072b05c2d9db1589a07e20f38902b1ec
!pip install -U git+https://github.com/huggingface/safetensors.git@f5839b6aee407652aa3078d91206b618dd84e3c2

!pip install sacrebleu
!pip install pytest-playwright
!playwright install
!pip install matplotlib
!pip install pillow
!pip install torchvision
!pip install lpips

!playwright install-deps  

!pip install -U numpy
!pip install tensorboard

  Cloning https://github.com/huggingface/transformers.git (to revision 13493215abceafc1653af88b045120014fb4c1fc) to /tmp/pip-req-build-8h8jkr17
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-8h8jkr17
  Running command git rev-parse -q --verify 'sha^13493215abceafc1653af88b045120014fb4c1fc'
  Running command git fetch -q https://github.com/huggingface/transformers.git 13493215abceafc1653af88b045120014fb4c1fc
  Running command git checkout -q 13493215abceafc1653af88b045120014fb4c1fc
  Resolved https://github.com/huggingface/transformers.git to commit 13493215abceafc1653af88b045120014fb4c1fc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.6/447.6 kB 12.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!apt install zip -y
!rm -rf data-rb-color
!mkdir -p data-rb-color
!wget "https://www.dropbox.com/scl/fi/vd0ypt9mo9oh0p9tf90h3/dataset-rb-color-fixed.zip?rlkey=bieseudpp5pzko5j4u1n67phq&dl=1" -O model.zip
!unzip model.zip -d data-rb-color

!mkdir -p data-rb-validate

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  unzip
The following NEW packages will be installed:
  unzip zip
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 350 kB of archives.
After this operation, 930 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 unzip amd64 6.0-26ubuntu3.2 [175 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 zip amd64 3.0-12build2 [176 kB]
Fetched 350 kB in 1s (450 kB/s)m
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package unzip.
(Reading database ... 34390 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-26ubuntu3.2_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 11%] [######................................

In [3]:
from datasets import load_from_disk

dataset = load_from_disk('data-rb-color')
dataset = dataset.train_test_split(test_size=0.00004) 


In [4]:
print(dataset['train'][0])

{'svg': '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="393" height="852" viewBox="0 0 393 852"><g id="html1"><g data-tag="head" id="head1" data-z-index="auto" data-stacking-context="true" aria-owns="script1"><g data-tag="script" id="script1" data-z-index="auto" data-stacking-context="true"/></g><g data-tag="body" id="body1" data-z-index="auto" data-stacking-context="true" role="document" aria-owns="style1"><g data-stacking-layer="rootBackgroundAndBorders"><rect width="377" height="836" x="8" y="8" fill="rgb(117, 232, 78)"/></g><g data-tag="style" id="style1" data-z-index="auto" data-stacking-context="true"/></g></g></svg>', 'html': '<body></body>\n\n<style>\n\n        body {\n            background-color: #75e84e;\n        }\n    \n</style>'}


In [6]:
from transformers import T5TokenizerFast
from tokenizers.processors import TemplateProcessing

tokenizer = T5TokenizerFast(tokenizer_file="data-rb-color/tokenizer.json",  
                                padding=True, 
                                eos_token="</s>",unk_token="<unk>", 
                                pad_token="<pad>")

post_processor = TemplateProcessing(
    single="$A </s>",
    pair="$A </s> $B </s>",
    special_tokens=[("</s>", tokenizer.eos_token_id)]
)

tokenizer._tokenizer.post_processor = post_processor

In [7]:
tokenized_output = tokenizer.encode(dataset['train'][0]['svg'], add_special_tokens=False)
unk_token_id = tokenizer.unk_token_id
if unk_token_id in tokenized_output:
    print("Alert: The tokenized SVG contains unknown tokens.")
else:
    print("No unknown tokens found in the SVG.")

tokenized_output = tokenizer.encode(dataset['train'][0]['html'], add_special_tokens=False)
unk_token_id = tokenizer.unk_token_id
if unk_token_id in tokenized_output:
    print("Alert: The tokenized HTML contains unknown tokens.")
else:
    print("No unknown tokens found in the HTML.")

No unknown tokens found in the SVG.
No unknown tokens found in the HTML.


In [8]:
MAX_SEQUENCE_LENGTH = 512

def get_token_lengths(examples):
    inputs = tokenizer(
        examples['svg'],
        truncation=False,  # Don't truncate yet
        padding=False,     # Don't pad yet
        return_length=True,
    )
    labels = tokenizer(
        examples['html'],
        truncation=False,
        padding=False,
        return_length=True,
    )

    inputs["input_length"] = inputs["length"]
    inputs["labels"] = labels["input_ids"]
    inputs["label_length"] = labels["length"]

    inputs.pop("length", None)

    return inputs

tokenized_data = dataset.map(get_token_lengths, batched=True)

def filter_function(example):
    return example['input_length'] <= MAX_SEQUENCE_LENGTH and example['label_length'] <= MAX_SEQUENCE_LENGTH

filtered_data = tokenized_data.filter(filter_function)

def tokenize_examples(examples):
    inputs = tokenizer(
        examples['svg'],
        max_length=MAX_SEQUENCE_LENGTH,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
    )
    labels = tokenizer(
        examples['html'],
        max_length=MAX_SEQUENCE_LENGTH,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
    )

    labels = labels['input_ids']
    labels[labels == tokenizer.pad_token_id] = -100
    inputs["labels"] = labels
    

    return inputs

filtered_data = filtered_data.map(get_token_lengths, batched=True, remove_columns=dataset["train"].column_names)

print(filtered_data)

Map:   0%|          | 0/99996 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Filter:   0%|          | 0/99996 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/99996 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'input_length', 'labels', 'label_length'],
        num_rows: 99996
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'input_length', 'labels', 'label_length'],
        num_rows: 4
    })
})


In [9]:
import numpy as np
from utils.similarity import calculate_metrics
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import torch

def add_image_to_tensorboard(name, step, img_path):
    image = Image.open(img_path)
    image = image.convert('RGB')
    image_array = np.array(image)
    image_tensor = torch.from_numpy(image_array)
    image_tensor = image_tensor.permute(2, 0, 1)
    image_tensor = image_tensor.float() / 255.0
    
    writer = SummaryWriter(log_dir=log_dir)
    writer.add_image(name, image_tensor, step)
    
def add_text_to_tensorboard(name, step, text):
    writer = SummaryWriter(log_dir=log_dir)
    writer.add_text(name, text, step)

def postprocess_text(preds, labels):
    preds = [pred.strip().replace('<unk>', '') for pred in preds]
    labels = [[label.strip().replace('<unk>', '')] for label in labels]

    return preds, labels

def create_compute_metrics(trainer):
    def compute_metrics(eval_predictions):
        predictions, labels = eval_predictions
        if isinstance(predictions, tuple):
            predictions = predictions[0]
            
        predictions = predictions[-4:]
        labels = labels[-4:]
            
        decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
        decoded_predictions, decoded_labels = postprocess_text(decoded_predictions, decoded_labels)
    
        similarity_scores = []
        perceptual_losses = []
        index = 1
        for prediction, label in zip(decoded_predictions, decoded_labels):
            add_text_to_tensorboard(f'valid_{index}_label_text', trainer.state.global_step, label[0])
            add_text_to_tensorboard(f'valid_{index}_prediction_text', trainer.state.global_step, prediction)
            
            metrics = calculate_metrics(prediction, label[0])
            
            if metrics is not None:
                similarity_scores.append(metrics['similarity'])
                perceptual_losses.append(metrics['perceptual_loss'])
                
                add_image_to_tensorboard(f'valid_{index}_expectation', trainer.state.global_step, metrics['expected_screenshot_path'])
                add_image_to_tensorboard(f'valid_{index}_prediction', trainer.state.global_step, metrics['predicted_screenshot_path'])
            
            index += 1
        
        result = {
            'similarity': float(np.mean(similarity_scores)),
            'perceptual_loss': float(np.mean(perceptual_losses)),
        }
    
        result = {k: round(v, 4) for k, v in result.items()}
        return result
    
    return compute_metrics

In [10]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="google-t5/t5-small")

In [11]:
from transformers import AutoModelForSeq2SeqLM, AutoConfig

config = AutoConfig.from_pretrained(
    "google-t5/t5-small",
    vocab_size=len(tokenizer),
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id = tokenizer.eos_token_id,
    decoder_start_token_id = tokenizer.pad_token_id,
)

model = AutoModelForSeq2SeqLM.from_config(config)

model.resize_token_embeddings(len(tokenizer))

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Embedding(32228, 512)

In [12]:
from transformers import GenerationConfig

generation_config = GenerationConfig.from_model_config(model.config)
generation_config._from_model_config = False
generation_config.max_new_tokens = 512

generation_config

GenerationConfig {
  "decoder_start_token_id": 3,
  "eos_token_id": 1,
  "max_new_tokens": 512,
  "pad_token_id": 3
}

In [13]:
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_output_dir = f"model-{timestamp}"
log_dir = f'{model_output_dir}/runs'

In [14]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=model_output_dir,
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # gradient_accumulation_steps=16,
    # gradient_checkpointing=True,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    logging_steps=25,
    logging_dir=log_dir,
    predict_with_generate=True,
    # load_best_model_at_end=True,
    fp16=True,
    generation_config=generation_config,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=filtered_data["train"],
    eval_dataset=filtered_data["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=None,
)

trainer.compute_metrics = create_compute_metrics(trainer)

trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss,Similarity,Perceptual Loss,Runtime,Samples Per Second,Steps Per Second
1000,1.508800,1.464997,0.668000,0.519500,19.389500,0.206000,0.052000
2000,1.422600,1.422667,0.668000,0.519500,20.359000,0.196000,0.049000
3000,1.379600,1.318705,0.668000,0.519500,18.162000,0.220000,0.055000
4000,1.299300,1.264981,0.734300,0.488200,19.750800,0.203000,0.051000
5000,1.219300,1.186139,0.668000,0.519500,19.079000,0.210000,0.052000
6000,1.168400,1.135667,0.849700,0.446900,18.215800,0.220000,0.055000
7000,1.119100,1.095600,0.668000,0.519500,17.580700,0.228000,0.057000
8000,1.078600,1.039257,0.705500,0.516400,18.647600,0.215000,0.054000
9000,1.025600,1.011438,0.879000,0.438400,17.693900,0.226000,0.057000
10000,0.962800,0.949884,0.720700,0.515600,19.008100,0.210000,0.053000


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=62500, training_loss=0.618044246498108, metrics={'train_runtime': 3753.176, 'train_samples_per_second': 266.43, 'train_steps_per_second': 16.653, 'total_flos': 2.4905736820752384e+16, 'train_loss': 0.618044246498108, 'epoch': 10.0})

In [16]:
import os

trainer.save_model(os.path.join(model_output_dir, "final"))
tokenizer.save_pretrained(os.path.join(model_output_dir, "final"))

('model-2024-12-06_13-55-57/final/tokenizer_config.json',
 'model-2024-12-06_13-55-57/final/special_tokens_map.json',
 'model-2024-12-06_13-55-57/final/tokenizer.json')

In [17]:
from transformers import T5ForConditionalGeneration

text = dataset['test'][0]['svg']

inputs = tokenizer(text, return_tensors="pt").input_ids
inputs = inputs.to('cuda:0')

model = T5ForConditionalGeneration.from_pretrained(os.path.join(model_output_dir, "final"))
model.to('cuda:0')
model.generation_config = generation_config

outputs = model.generate(inputs)
# outputs = model.generate(inputs, max_new_tokens=16_384, do_sample=True, top_k=30, top_p=0.95)

translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(dataset['test'][0]['html'])
print(translation)


<body></body>

<style>

        body {
            background-color: #5245d1;
        }
    
</style>
<body></body> <style> body { background-color: #5245d1; } </style>
